In [18]:
import json
import pickle
import numpy as np
from operator import itemgetter
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

In [19]:
# Read files
with open('clean_data_leap/sorted_diag_list.pkl', 'rb') as f:
        sorted_diag_list = pickle.load(f)

with open('clean_data_leap/sorted_drug_list.pkl', 'rb') as f:
        sorted_drug_list = pickle.load(f)

In [20]:
# Read models & generate probability matrix
prob_list = list()
for i in range(50):
    lr = joblib.load('logistic_models_top538/model' + str(i) + '.m')
    diag_iden_matrix = np.eye(len(sorted_diag_list[:2001]))
    diag_predict = lr.predict_proba(diag_iden_matrix)
    prob = list()
    for predict in diag_predict:
        prob.append(predict[1])
    prob_list.append(prob)
prob_matrix = np.matrix(prob_list)

In [21]:
# Drug recommendation for each disease
count = 0
single_rec = {}
with open('recommendation_leap/single_rec_50.txt', "w") as f:
    for j in range(np.size(prob_matrix, axis=1)):
        diag = prob_matrix[:,j].tolist()
        drug_prob = {}
        for i in range(len(diag)):
            drug_prob[i] = diag[i][0]
        sorted_prob = sorted(drug_prob.items(), key=itemgetter(1), reverse=True)
        f.write('%s:' %(sorted_diag_list[count][0]))
        single_rec[sorted_diag_list[count][0]] = sorted_prob
        for drug in sorted_prob:
            f.write('%s,%.4f; ' %(sorted_drug_list[drug[0]][0], drug[1]))
        f.write('\n------------------------------------------------------\n')
        count += 1

In [22]:
# Save single disease drug recommendation dict
with open('recommendation_leap/single_rec_50.pkl', 'wb') as f:
        pickle.dump(single_rec, f)